In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, metrics, linear_model, ensemble, tree, grid_search, externals
from time import time
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import externals, metrics
import cPickle as pickle
from __future__ import division
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns

C:\Analytics Software\Anaconda\anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Analytics Software\Anaconda\anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
%cd C:\Users\benli\Documents\Project

C:\Users\benli\Documents\Project


In [3]:
reviews_train = pd.read_pickle('reviews_train.pkl')

In [4]:
# Making the dataset only for restaurants
reviews_train = reviews_train[reviews_train['category'] == 'Restaurants']

In [5]:
# Making the dataset only for Nevada state
reviews_train = reviews_train[reviews_train['state'] == 'NV']

In [6]:
# clean reviews by keeping only alphabetic characters and exclamation marks
text_cleaned = [re.sub(r'!', ' exclamation', text) for text in reviews_train['text']]
text_cleaned = [text.lower() for text in text_cleaned]
text_cleaned = [re.sub(r'[^a-z]', ' ', text) for text in text_cleaned]
reviews_train['text_cleaned'] = text_cleaned

In [7]:
# create document-term matrix using the cleaned reviews
vectorizer = CountVectorizer(min_df = 100)
dtm = vectorizer.fit_transform(reviews_train['text_cleaned'])
dtm.shape

(505807, 10290)

To calculate sentiment scores, first calculate the percentage of time a word "spends" on each document, defined as its usage in each individual document divided by its total usage across all documents (i.e., the column sum), thus creating a modified document-term matrix whose elements are all "percentage frequencies."

In [8]:
# calculate the total frequency across all documents for each word (i.e., the column sums)
freq = csr_matrix.sum(dtm, 0)

# devide the individual frequency by the total frequency to calculate the percentage of time a word "spends" on each document
freq_inverse = np.repeat(1, freq.shape[1]) / freq
freq_inverse = csr_matrix(freq_inverse)
dtm_prc = dtm.multiply(freq_inverse)

Then, for each word, multiply its percentage frequency in each document by the number of stars the reviewer ultimately assigned to the business (normalized to [-1, 1]), and sum these weighted frequencies across all documents to arrive at the word's sentiment score.

As an example, say the word "good" shows up 5 times in a 5-star review, 4 times in a 4-star review, ..., and 1 time in a 1-star review, its sentiment score would be (5 * 1 + 4 * 0.5 + 3 * 0 + 2 * (-0.5) + 1 * (-1)) / (5 + 4 + 3 + 2 + 1) = 0.33, indicating moderately postive sentiment.

As a couple extreme examples, if a word only appears in 5-star reviews, its sentiment score would be 1, and if it only appears in 1-star reviews, its score would be -1.

What about highly frequent words that don't mean much (e.g., stopwords)? As shown in the first example, showing up everywhere will dilute a word's overall effect. In an extreme example, if a word's usage is evenly distributed across all documents, its final score would be 0, indicating abosulately neutral effect.

In [9]:
# calculate the sentiment score for each word by weighting its percentage frequency by the number of stars
# first normalize the star ratings to [-1, 1]:
def normalization(series):
    return (2 * series - (series.max() + series.min())) / (series.max() - series.min())
stars_norm = normalization(reviews_train['stars'])

# calculate word scores using the normlized star ratings
word_score = dtm_prc.T * stars_norm
sns.distplot(word_score, kde = False)

In [10]:
# combine with the terms
word2score = dict(zip(vectorizer.get_feature_names(), word_score))

In [11]:
# calculate sentiment scores for each reviews
review_score = dtm * np.atleast_2d(word_score).T

# divide the scores by the total number of words in each document
dtm_row_sum = csr_matrix.sum(dtm, 1)
review_score = review_score / dtm_row_sum

In [12]:
# for reviews that don't include any words in the dtm, replace the 0 scores with NAs
review_score[dtm_row_sum == 0] = np.nan
print 'the number of reviews with NA scores:', sum(pd.isnull(review_score))[0, 0]

the number of reviews with NA scores: 94


In [13]:
# combine scores with the reviews
reviews_train['score'] = review_score

In [14]:
# Removing na
reviews_train_excl_na = reviews_train.dropna(subset = ['score'])

In [15]:
# create input and output variables
score = reviews_train_excl_na['score']
score = np.atleast_2d(score).T # the feature set needs to be 2-dimensional
stars = reviews_train_excl_na['stars']

In [16]:
scorer_mae = metrics.make_scorer(metrics.mean_absolute_error, greater_is_better = False)

<h2 style="font-family:Times New Roman;">Logistic Regression</h2>

In [17]:
logit_clf = linear_model.LogisticRegression(random_state = 2014)

t0 = time()
logit_scores_mae = cross_validation.cross_val_score(logit_clf, score, stars, cv = 10, scoring = scorer_mae)
t1 = time()

In [18]:
print 'logistic regression using mean-absolute-error scoring function'
print 'time used: %d seconds' % (t1 - t0)
print 'scores: %0.3f (+/-%0.03f)' % (logit_scores_mae.mean(), logit_scores_mae.std())

logistic regression using mean-absolute-error scoring function
time used: 18 seconds
scores: -0.712 (+/-0.003)


The logistic model finished quickly with decent results, indicating the computed sentiment scores can linearly separate the data to some extent. The mean scores of the 10-fold cross-validation is 0.712 (the negative sign is added automatically indicating the new scoring function is a loss function), suggesting on average the predicted ratings are within 1-star away from the actual.

<h2 style="font-family:Times New Roman;">Baggigng Trees</h2>

In [19]:
bag_clf = ensemble.BaggingClassifier(tree.DecisionTreeClassifier(), n_estimators = 100, n_jobs = -1, random_state = 2014)

t0 = time()
bag_scores_mae = cross_validation.cross_val_score(bag_clf, score, stars, cv = 10, scoring = scorer_mae)
t1 = time()

In [20]:
print 'bagging trees using mean-absolute-error scoring function'
print 'time used: %d seconds' % (t1 - t0)
print 'scores: %0.3f (+/-%0.03f)' % (bag_scores_mae.mean(), bag_scores_mae.std())

bagging trees using mean-absolute-error scoring function
time used: 1257 seconds
scores: -0.783 (+/-0.004)


<h2 style="font-family:Times New Roman;">Gradient boost trees</h2>

In [21]:
# gradient boost trees
param_grid = {'learning_rate': [.01, .1], 'max_depth': [1, 5]}
gbm_clf = grid_search.GridSearchCV(ensemble.GradientBoostingClassifier(n_estimators = 100, random_state = 2014),
                                   param_grid, cv = 10, scoring = scorer_mae, n_jobs = -1)

t0 = time()
gbm_clf.fit(score, stars)
t1 = time()

In [22]:
print 'gradient boost trees using mean-absolute-error scoring function'
print 'time used: %d seconds' % (t1 - t0)
print 'best score:', gbm_clf.best_score_, gbm_clf.best_params_
print 'grid scores:'
for scores in gbm_clf.grid_scores_:
    print 'scores: %0.3f (+/-%0.03f)' % (scores[2].mean(), scores[2].std()), scores[0]

gradient boost trees using mean-absolute-error scoring function
time used: 3241 seconds
best score: -0.594242188751 {'learning_rate': 0.1, 'max_depth': 1}
grid scores:
scores: -0.627 (+/-0.004) {'learning_rate': 0.01, 'max_depth': 1}
scores: -0.595 (+/-0.003) {'learning_rate': 0.01, 'max_depth': 5}
scores: -0.594 (+/-0.003) {'learning_rate': 0.1, 'max_depth': 1}
scores: -0.595 (+/-0.003) {'learning_rate': 0.1, 'max_depth': 5}


<h2 style="font-family:Times New Roman;">Saving the winning model</h2>

In [23]:
# save the winning model
# Please save the model with the highest accuracy
externals.joblib.dump(gbm_clf, 'gbm_clf.pkl')

['gbm_clf.pkl']